In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from darts import TimeSeries

df = pd.read_csv('../csv/Ming25Jun67_total_order.csv')
df.tail()

In [ ]:
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])
# make timeseries Object
series = TimeSeries.from_dataframe(df,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'],
                                   freq="D")

In [ ]:
# Function to replace outliers with the mean of the rest of the values
from scipy.stats import zscore
from darts import TimeSeries

def replace_outliers_with_mean(df, column_name):
    # Calculate Z-scores
    z_scores = zscore(df[column_name])
    
    # Identify outliers (using a threshold of 3 for Z-score)
    outliers = np.abs(z_scores) > 3
    
    # Calculate mean of non-outliers
    mean_non_outliers = df.loc[~outliers, column_name].mean()
    
    # Replace outliers with the mean of non-outliers
    df.loc[outliers, column_name] = mean_non_outliers
    
    return df

# Replace outliers in 'gmv' column
df_remove_out = replace_outliers_with_mean(df, 'unique_order_count')
series_remove_outlier = TimeSeries.from_dataframe(df_remove_out,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'],
                                   freq="D")
series = series_remove_outlier

In [ ]:
# cut_off = pd.Timestamp('2024-04-01')
cut_off = 0.8
training, validation =series.split_before(cut_off) # 1 month

In [ ]:
validation_df = validation.pd_dataframe()
training_df = training.pd_dataframe()
print(training_df.tail())
print(validation_df.head())
print(training_df.shape)
print(validation_df.shape)

In [ ]:
from darts.models import Theta
from darts.metrics import mape, mae, mse, mase
from math import sqrt
from darts.utils.utils import ModelMode, SeasonalityMode

parameters = {
    "theta": [0.5, 1, 1.5, 2, 2.5, 3],
    "season_mode":[SeasonalityMode.MULTIPLICATIVE, SeasonalityMode.ADDITIVE],
    "seasonality_period":list(range(1,31))
}

best_parameter = Theta.gridsearch(
    parameters=parameters,
    series = series, #note (training + validation) X only training set!!!
    start=cut_off,
    forecast_horizon=7,
    stride=1,          
    last_points_only=False,  
    metric=mse,
    verbose=-1
)

In [ ]:
print(best_parameter)

In [ ]:
best_model = best_parameter[0]
best_model.fit(training)

In [ ]:
# from darts.metrics import mape, mae, mse, mase
# from math import sqrt

# backtest_errors = best_model.backtest(
#      series,
#      start=pd.Timestamp('2024-05-01'),          # เริ่มต้นที่ 90% ของข้อมูล
#      forecast_horizon=7, #จำนวนก้าวการพยากรณ์ในอนาคตที่ต้องการ
#      stride=1,           #ช่วงการเลื่อนในการทดสอบแต่ละครั้ง
#      last_points_only=False,  #  ให้แสดงเฉพาะจุดสิ้นสุดของการพยากรณ์หรือไม่
#      metric=mse,
#      reduction=np.mean           #ฟังก์ชันการลดรูปผลลัพธ์ (เช่น mean การหาเฉลี่ย)
# )
# print(f'backtest_errors MSE: {backtest_errors}')

In [ ]:
# from darts.metrics import mape, mae, mse, mase
# from math import sqrt

# backtest_errors = best_model.backtest(
#      series,
#      start=pd.Timestamp('2024-05-01'),          # เริ่มต้นที่ 90% ของข้อมูล
#      forecast_horizon=7, #จำนวนก้าวการพยากรณ์ในอนาคตที่ต้องการ
#      stride=1,           #ช่วงการเลื่อนในการทดสอบแต่ละครั้ง
#      last_points_only=False,  #  ให้แสดงเฉพาะจุดสิ้นสุดของการพยากรณ์หรือไม่
#      metric=mape,
#      reduction=np.mean           #ฟังก์ชันการลดรูปผลลัพธ์ (เช่น mean การหาเฉลี่ย)
# )
# print(f'backtest_errors MAPE: {backtest_errors}')

In [ ]:
# best_model.fit(training)

In [ ]:
# result from training , validation only
prediction = best_model.predict(len(validation))

mape_score = mape(validation, prediction)
print(f'score MAPE: {round(mape_score, 4)}')
print(f'score ACC: {100-(round(mape_score, 4))}')

mse_score = mse(validation, prediction)
print(f'score MSE: {round(mse_score, 4)}')

print(f'score RMSE: {round(sqrt(mse_score), 4)}')

mae_score = mae(validation, prediction)
print(f'score MAE: {round(mae_score, 4)}')

In [ ]:
# validation_rescale = scaler.inverse_transform(validation)
# prediction_rescale = scaler.inverse_transform(prediction)

# mape_score = mape(validation_rescale, prediction_rescale)
# print(f'score MAPE: {round(mape_score, 4)}')
# print(f'score ACC: {100-(round(mape_score, 4))}')

# mse_score = mse(validation_rescale, prediction_rescale)
# print(f'score MSE: {round(mse_score, 4)}')

# print(f'score RMSE: {round(sqrt(mse_score), 4)}')

# mae_score = mae(validation_rescale, prediction_rescale)
# print(f'score MAE: {round(mae_score, 4)}')

In [ ]:
plt.figure(figsize=(20, 10))  # ปรับขนาดของภาพ (กว้าง x สูง)
##prediction
prediction.plot(label="prediction")
training.plot(label="training")
validation.plot(label="validation")

plt.legend()
plt.title('Train, Validation, and Prediction')
plt.xlabel('Date')
plt.ylabel('Order Unit')
plt.show()

In [ ]:
# predict future
best_model.fit(series)

In [ ]:
forecast = best_model.predict(7)
# forecast  = scaler.inverse_transform(forecast)
forecast = forecast.pd_dataframe().reset_index()
forecast.columns = ['Date', 'Total Order']
print(forecast.tail(7))

forecast.tail(7).to_csv('../forecast/25-06-67_theta.csv', index=False)

In [ ]:
best_model.save("../model/theta_Model_totalOrder_25Jun67.pkl")

# FourTheta